In [1]:
import gc, os, pickle
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [3]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
def variable_summaries(var, name):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.scalar_summary('mean/' + name, mean)
        
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.scalar_summary('stddev/' + name, stddev)
            
        tf.scalar_summary('max/' + name, tf.reduce_max(var))
        tf.scalar_summary('min/' + name, tf.reduce_min(var))
        
        tf.histogram_summary(name, var)

In [5]:
def dense_connected_layer(input_tensor, weight_shape, bias_shape, layer_name, activation_fun = tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            W = weight_variable(weight_shape)
            variable_summaries(W, layer_name + '/weights')
        with tf.name_scope('biases'):
            b = bias_variable(bias_shape)
            variable_summaries(b, layer_name + '/biases')
        
        with tf.name_scope('Wx_plus_b'):
            h = tf.matmul(input_tensor, W) + b
            tf.histogram_summary(layer_name + '/Wx_plus_b', h)
            
        with tf.name_scope('activation'): 
            h_ = activation_fun(h, )
            tf.histogram_summary(layer_name + '/activation', h_)
        
    return h_

In [6]:
def softmax_layer(input_tensor, layer_name):
    with tf.name_scope(layer_name):
        h_softmax = tf.nn.softmax(input_tensor)
        tf.histogram_summary(layer_name + '/softmax', h_softmax)
        
    return h_softmax

In [7]:
def cost_function(y_, y, reduction_indices = [1]):
    with tf.name_scope('cost_function'):
        cost = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices = reduction_indices))
        tf.scalar_summary('cost_function', cost)
    
    return cost

In [8]:
def accuracy_function(y_, y):
    with tf.name_scope('accuracy_function'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.scalar_summary('/accuracy_function', accuracy)
        
    return accuracy

In [9]:
def optimizer(y_, y, cost_function = cost_function, optimization_function = tf.train.AdamOptimizer,
              learning_rate = 1e-2, reduction_indices = [1]):
    optimize = optimization_function(learning_rate).minimize(cost_function(y_, y, reduction_indices = [1]))
    
    return optimize

In [10]:
def save_session(session, checkpoint_name = "checkpoint", save_path = "backup"):
    saver = tf.train.Saver()
    saver.save(sess, save_path + "/" + checkpoint_name + ".ckpt")

In [11]:
def restore_session(session, checkpoint_name = "checkpoint", save_path = "backup"):
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(save_path)
    temp = 0
    if ckpt and ckpt.model_checkpoint_path:
        try:
            saver.restore(session, ckpt.model_checkpoint_path)
            temp += 1
        finally:
            temp += 1
            
    if temp == 0:    
        return -1
    return temp

In [12]:
ITERATIONS = 10 ** 3
LEARNING_RATE = 1e-3

In [13]:
sess = tf.InteractiveSession()

In [14]:
x = tf.placeholder(tf.float32, shape = [None, 4])
y_ = tf.placeholder(tf.float32, shape = [None, 3])

In [15]:
layer1 = dense_connected_layer(input_tensor = x, weight_shape = [4, 32], bias_shape = [32], 
                               layer_name = "layer1")

In [16]:
layer2 = dense_connected_layer(input_tensor = layer1, weight_shape = [32, 64], bias_shape = [64], 
                               layer_name = "layer2")

In [17]:
layer3 = dense_connected_layer(input_tensor = layer2, weight_shape = [64, 3], bias_shape = [3], 
                               layer_name = "layer3")

In [18]:
y = softmax_layer(input_tensor = layer3, layer_name = "layer4")

In [19]:
optimize = optimizer(y_, y, cost_function = cost_function, learning_rate = LEARNING_RATE)

In [20]:
data = pd.read_csv("Iris.csv")
data_dummy = pd.get_dummies(data["Species"])
data = data.join(data_dummy)
data = data.drop(["Species"], axis = 1)
data = data.iloc[np.random.permutation(len(data))]
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Iris-setosa,Iris-versicolor,Iris-virginica
96,97,5.7,2.9,4.2,1.3,0.0,1.0,0.0
88,89,5.6,3.0,4.1,1.3,0.0,1.0,0.0
32,33,5.2,4.1,1.5,0.1,1.0,0.0,0.0
117,118,7.7,3.8,6.7,2.2,0.0,0.0,1.0
8,9,4.4,2.9,1.4,0.2,1.0,0.0,0.0


In [21]:
features = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]
label = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]

In [22]:
train_features = data[features].values
train_features = train_features / np.amax(train_features)
train_labels = data[label].values

In [23]:
train_size = int(0.7 * train_features.shape[0])

In [24]:
validation_features = train_features[train_size:]
validation_labels = train_labels[train_size:]

train_features = train_features[:train_size]
train_labels = train_labels[:train_size]

In [25]:
accuracy = accuracy_function(y_, y)

In [26]:
if restore_session(sess) < 0:
    sess.run(tf.initialize_all_variables())

In [27]:
for i in range(ITERATIONS):
    optimize.run(feed_dict={x : train_features, y_ : train_labels})
    
    if i % 10.0 == 0.0:
        save_session(sess)
        
accu = accuracy.eval(feed_dict = {x : train_features, y_ : train_labels})
print("Training Accuracy = " + str(accu))

Training Accuracy = 0.980952


In [28]:
save_session(sess)

In [29]:
accu = accuracy.eval(feed_dict = {x : validation_features, y_ : validation_labels})
print("Validation Accuracy = " + str(accu))

Validation Accuracy = 1.0
